In [1]:
#clear out everything
%reset -f
import math
from sympy import * #import everything
from __future__ import division
from sympy.abc import *
import numpy as np
import matplotlib.pyplot as plt
import platform
from IPython.display import display
from UnitConversion import * #custom .py file with unit conversions
init_printing() #print prettier output
%matplotlib inline 

# Estimating Heat Load in the Room

Energy entering room: Furnace, Preheater, Fuel

Energy leaving room: Hot Exhaust Gases

### Advection

https://en.wikipedia.org/wiki/Heat_transfer

Heat transfer by transferring matter, energy is moved by the physical transfer of a hot or cold object/material from one place to another. This can be described by the formula:

\begin{equation*}
Q_{adv} = v \rho c_{p} \Delta T 
\quad\mathrm{(W/m^{2})}
\tag{1}
\end{equation*}

where $Q$ is heat flux (W/m²), $\rho$ is density (kg/m³), $c_{p}$ is heat capacity at constant pressure (J/kg·K), $\Delta T$ is the change in temperature (K), and $v$ is velocity (m/s).

On a total basis, not per unit area, heat transfer becomes:

\begin{equation*}
q_{adv}=\dot{m} c_{p} \Delta T
\quad\mathrm{(W)}
\tag{2}
\end{equation*}

where $\dot{m}$ is mass flow rate in (kg/s).

\begin{equation*}
\Delta T=T_f - T_i
\quad\mathrm{(K)}
\tag{3}
\end{equation*}

In [2]:
#declare symbols
var(' q_adv c_p T_f T_i ')
deltaT = Symbol('\Delta T') #in Jupyter notebook, you can set the symbol name to be the LaTeX of what you want
mdot_gas = Symbol('\dot{m}_{gas}')

#equations, symbolic equations in SymPy are not represented by = or ==, but by Eq
eq1 = Eq(q_adv, mdot_gas*c_p*deltaT)
eq2 = Eq(deltaT, T_f - T_i) #temperature difference between inlet and outlet of room
display(eq1)
display(eq2)

In [3]:
r1 = solve(eq2,deltaT,dict=1) #solve eq2 for deltaT, output as dictionary object
r2 = eq1.subs(r1[0]) #substitute the first (and only) item in the dictionary object into eq1
display(r2)

In [4]:
#values in preferred units
_T_f = 750 #F, final temperature
_T_i = 120 #F, initial temperature
_c_p = 1340 #J/kgK, specific heat (http://www.pipeflowcalculations.com/tables/flue-gas.php)
_mdot_gas = 0.95 #kg/s, mass flow rate of gas exiting the room (assumes addition of dilution air at ~4:1 kg/kg)

#corrected units for equation
_T_f = funFtoK(_T_f) #K
_T_i = funFtoK(_T_i) #K

#substitute values for variables
subsList = { T_f:_T_f, T_i:_T_i, c_p:_c_p, mdot_gas:_mdot_gas }
display(subsList)

#calculate q_adv
temp = r2.subs(subsList)
display(temp)
_q_adv = temp.args[1]
print("q_adv = %.0f" %(_q_adv/1000) + " kW, heat exiting room through exhaust") #show N digits after decimal point

q_adv = 446 kW, heat exiting room through exhaust


### How much energy is entering the room?

#### From fuel:
http://www.engineeringtoolbox.com/fuels-higher-calorific-values-d_169.html

The calorific value of a fuel is the quantity of heat produced by its combustion - at constant pressure and under "normal" ("standard") conditions (i.e. to 0oC and under a pressure of 1013 mbar).

The combustion process generates water vapor and certain techniques may be used to recover the quantity of heat contained in this water vapor by condensing it.

Higher Calorific Value (or Gross Calorific Value - GCV, or Higher Heating Value - HHV) -  the water of combustion is entirely condensed and that the heat contained in the water vapor is recovered

Lower Calorific Value (or Net Calorific Value - NCV, or Lower Heating Value - LHV) - the products of combustion contains the water vapor and that the heat in the water vapor is not recovered

$HHV_{Diesel} = 44800 \quad(kJ/kg)$

$HHV_{Arabian\,Super\,Light} = 44769 \quad(kJ/kg)$

$HHV_{Arabian\,Heavy} = 43443 \quad(kJ/kg)$

(Arabian Super Light, Arabian Heavy values are from SRD ref B: 19247, 18677 BTU/lbm)

We don't have LHV values, and our conditions aren't standard, so we guess at a combustion efficiency, $\eta$ in (%).

Energy entering the room via the fuel $E_{in}$ should be:

\begin{equation*}
E_{in} = HHV \times \dot{m}_{fuel} \times \frac{\eta}{100}
\quad\mathrm{(W)}
\tag{4}
\end{equation*}

In [5]:
#declare symbols
var(' E_fuel HHV eta ')
mdot_fuel = Symbol('\dot{m}_{fuel}')

#equations, symbolic equations in SymPy are not represented by = or ==, but by Eq
eq3 = Eq(E_fuel, HHV*mdot_fuel*eta/100)
display(eq3)

#values in preferred units
_HHV = 44800*1000 #J/kg, higher heating value
_mdot_fuel = .0156 #kg/s, mass flow rate of fuel (max flow, 19 gal/hour)
_eta = 100 #%, heat transfer efficiency 65-85% for home furnaces, 100% is worst case for AC sizing (http://www.ems.psu.edu/~radovic/Chapter4.pdf)

#substitute values for variables
subsList.update({ HHV:_HHV, mdot_fuel:_mdot_fuel, eta:_eta })
display(subsList)

#calculate
temp = eq3.subs(subsList)
display(temp)
_E_fuel = temp.args[1]
print("E_fuel = %.0f" %(_E_fuel/1000) + " kW, heat entering room from fuel combustion") #show N digits after decimal point

E_fuel = 699 kW, heat entering room from fuel combustion


#### From Furnace, PreHeater:
Furnace is rated at 17.2 kW, so if it were running at max capacity...

Preheater is rated at 41.2 kW, so if it were running at max capacity...

(Not including the centrifuge because it should not run simultaneously with the rig)

#### Total

In [6]:
var(' E_total E_furnace E_preheater')
_E_furnace = 17.2*1000 #W
_E_preheater = 41.2*1000 #W

eq4 = Eq(E_total, E_fuel + E_furnace + E_preheater - q_adv)
display(eq4)

_E_total = eq4.subs({E_fuel:_E_fuel, q_adv:_q_adv, E_furnace:_E_furnace, E_preheater:_E_preheater})
display(_E_total)
print("E_total = %.0f" %(_E_total.args[1]/1000) + " kW, heat load into room") #show N digits after decimal point

E_total = 312 kW, heat load into room
